<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

#!pip install onnx onnxruntime


In [9]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
softmax_output_name = "SoftmaxOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensor
input_tensor = onnx.helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, [None, None])

# Create output tensor (final result after softmax operation)
softmax_output = onnx.helper.make_tensor_value_info(softmax_output_name, onnx.TensorProto.FLOAT, [None, None])

def create_softmax_model(axis=1):
    # Define Softmax node: Y = Softmax(X) with axis
    softmax_node = onnx.helper.make_node("Softmax", [input_name], [softmax_output_name], axis=axis)
    
    # Create the ONNX graph
    graph = onnx.helper.make_graph(
        nodes=[softmax_node],
        name=f"Softmax_axis_{axis}",
        inputs=[input_tensor],
        outputs=[softmax_output]
    )

    # Create the ONNX model (Softmax opset 13)
    model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)])
    onnx.checker.check_model(model)
    onnx.save(model, f"softmax_axis_{axis}.onnx")
    return ort.InferenceSession(f"softmax_axis_{axis}.onnx")

# Create sessions for axis=0 and axis=1
session_axis0 = create_softmax_model(axis=0)
session_axis1 = create_softmax_model(axis=1)

def do_softmax(x, axis=1):
    session = session_axis0 if axis==0 else session_axis1
    output = session.run(None, {input_name: x})

    x_f = np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '')
    o_f = np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '')

    print(f"X (axis={axis}) = {x_f}")
    print(f"Softmax(X) = {o_f}\n")


np.set_printoptions(precision=8, floatmode='fixed')

#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

print("## Nominal Cases\n")

# Case 1: 1x2 vector
x = np.array([[9.5, 35.7]], dtype=np.float32)
do_softmax(x, axis=1)

# Case 2: 2x3 matrix
x = np.array([[1.0, 2.0, 3.0],
              [4.0, 5.0, 6.0]], dtype=np.float32)
do_softmax(x, axis=0)
do_softmax(x, axis=1)

#--------------------------------------------------------------------
# Special values: +inf, -inf, NaN
#--------------------------------------------------------------------

print("## Special Value Cases\n")

# Case +inf
x = np.array([[1, 2, 3],
              [4, 5, np.inf]], dtype=np.float32)
do_softmax(x, axis=0)
do_softmax(x, axis=1)

# Case -inf
x = np.array([[1, 2, 3],
              [4, 5, -np.inf]], dtype=np.float32)
do_softmax(x, axis=0)
do_softmax(x, axis=1)

# Case NaN
x = np.array([[1, 2, 3],
              [4, 5, np.nan]], dtype=np.float32)
do_softmax(x, axis=0)
do_softmax(x, axis=1)


## Nominal Cases

X (axis=1) = [[ 9.50000000,35.70000076]]
Softmax(X) = [[4.18296517e-12,1.00000000e+00]]

X (axis=0) = [[1.00000000,2.00000000,3.00000000], [4.00000000,5.00000000,6.00000000]]
Softmax(X) = [[0.04742587,0.04742587,0.04742587], [0.95257413,0.95257413,0.95257413]]

X (axis=1) = [[1.00000000,2.00000000,3.00000000], [4.00000000,5.00000000,6.00000000]]
Softmax(X) = [[0.09003057,0.24472848,0.66524094], [0.09003057,0.24472848,0.66524094]]

## Special Value Cases

X (axis=0) = [[1.00000000,2.00000000,3.00000000], [4.00000000,5.00000000,       inf]]
Softmax(X) = [[0.04742587,0.04742587,       nan], [0.95257413,0.95257413,       nan]]

X (axis=1) = [[1.00000000,2.00000000,3.00000000], [4.00000000,5.00000000,       inf]]
Softmax(X) = [[0.09003057,0.24472848,0.66524094], [       nan,       nan,       nan]]

X (axis=0) = [[1.00000000,2.00000000,3.00000000], [4.00000000,5.00000000,      -inf]]
Softmax(X) = [[0.04742587,0.04742587,1.00000000], [0.95257413,0.95257413,0.00000000]]

X (a

In [5]:
#--------------------------------------------------------------------
# Additional special cases (tests 6 to 20)
#--------------------------------------------------------------------

print("\n## Test 6: 3 flottants normaux + +inf")
x_test_6 = np.array([[1.0, 2.0, 3.0, np.inf]], dtype=np.float32)
do_softmax(x_test_6, axis=1)

print("\n## Test 7: 3 flottants normaux + -inf")
x_test_7 = np.array([[1.0, 2.0, 3.0, -np.inf]], dtype=np.float32)
do_softmax(x_test_7, axis=1)

print("\n## Test 8: 3 flottants normaux + NaN")
x_test_8 = np.array([[1.0, 2.0, 3.0, np.nan]], dtype=np.float32)
do_softmax(x_test_8, axis=1)

print("\n## Test 9: 3 flottants égaux à -0")
x_test_9 = np.array([[-0.0, -0.0, -0.0]], dtype=np.float32)
do_softmax(x_test_9, axis=1)

print("\n## Test 10: 3 flottants -0, 0, 0")
x_test_10 = np.array([[-0.0, 0.0, 0.0]], dtype=np.float32)
do_softmax(x_test_10, axis=1)

print("\n## Test 11: 3 flottants normaux + +inf et -inf")
x_test_11 = np.array([[1.0, 2.0, 3.0, np.inf, -np.inf]], dtype=np.float32)
do_softmax(x_test_11, axis=1)

print("\n## Test 12: 3 flottants normaux + +inf et NaN")
x_test_12 = np.array([[1.0, 2.0, 3.0, np.inf, np.nan]], dtype=np.float32)
do_softmax(x_test_12, axis=1)

print("\n## Test 13: 3 flottants normaux + +inf et +0")
x_test_13 = np.array([[1.0, 2.0, 3.0, np.inf, 0.0]], dtype=np.float32)
do_softmax(x_test_13, axis=1)

print("\n## Test 14: 3 flottants normaux + +inf et -0")
x_test_14 = np.array([[1.0, 2.0, 3.0, np.inf, -0.0]], dtype=np.float32)
do_softmax(x_test_14, axis=1)

print("\n## Test 15: 3 flottants normaux + -inf et NaN")
x_test_15 = np.array([[1.0, 2.0, 3.0, -np.inf, np.nan]], dtype=np.float32)
do_softmax(x_test_15, axis=1)

print("\n## Test 16: 3 flottants normaux + -inf et +0")
x_test_16 = np.array([[1.0, 2.0, 3.0, -np.inf, 0.0]], dtype=np.float32)
do_softmax(x_test_16, axis=1)

print("\n## Test 17: 3 flottants normaux + -inf et -0")
x_test_17 = np.array([[1.0, 2.0, 3.0, -np.inf, -0.0]], dtype=np.float32)
do_softmax(x_test_17, axis=1)

print("\n## Test 18: 3 flottants normaux <0 + -0 + +0")
x_test_18 = np.array([[-1.0, -2.0, -3.0, -0.0, 0.0]], dtype=np.float32)
do_softmax(x_test_18, axis=1)

print("\n## Test 19: 3 flottants normaux >0 + -0 + +0")
x_test_19 = np.array([[1.0, 2.0, 3.0, -0.0, 0.0]], dtype=np.float32)
do_softmax(x_test_19, axis=1)

print("\n## Test 20: 1 flottant >0, 1 flottant <0, -0, +0, 2")
x_test_20 = np.array([[1.0, -1.0, -0.0, 0.0, 2.0]], dtype=np.float32)
do_softmax(x_test_20, axis=1)


## Test 6: 3 flottants normaux + +inf
X (axis=1) = [[1.00000000,2.00000000,3.00000000,       inf]]
Softmax(X) = [[nan,nan,nan,nan]]


## Test 7: 3 flottants normaux + -inf
X (axis=1) = [[1.00000000,2.00000000,3.00000000,      -inf]]
Softmax(X) = [[0.09003057,0.24472848,0.66524094,0.00000000]]


## Test 8: 3 flottants normaux + NaN
X (axis=1) = [[1.00000000,2.00000000,3.00000000,       nan]]
Softmax(X) = [[nan,nan,nan,nan]]


## Test 9: 3 flottants égaux à -0
X (axis=1) = [[-0.00000000,-0.00000000,-0.00000000]]
Softmax(X) = [[0.33333334,0.33333334,0.33333334]]


## Test 10: 3 flottants -0, 0, 0
X (axis=1) = [[-0.00000000, 0.00000000, 0.00000000]]
Softmax(X) = [[0.33333334,0.33333334,0.33333334]]


## Test 11: 3 flottants normaux + +inf et -inf
X (axis=1) = [[1.00000000,2.00000000,3.00000000,       inf,      -inf]]
Softmax(X) = [[nan,nan,nan,nan,nan]]


## Test 12: 3 flottants normaux + +inf et NaN
X (axis=1) = [[1.00000000,2.00000000,3.00000000,       inf,       nan]]
Softmax(X) = [[na

Code for 3D dimension

In [10]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
softmax_output_name = "SoftmaxOutput"

def create_softmax_model(axis=1, rank=2):
    """
    Crée un modèle ONNX Softmax pour un tenseur de rang `rank` et un axe `axis`.
    """
    # Définition dynamique de la forme d'entrée et de sortie
    shape = [None] * rank
    input_tensor = onnx.helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, shape)
    softmax_output = onnx.helper.make_tensor_value_info(softmax_output_name, onnx.TensorProto.FLOAT, shape)

    # Crée le nœud Softmax
    softmax_node = onnx.helper.make_node(
        "Softmax",
        inputs=[input_name],
        outputs=[softmax_output_name],
        axis=axis
    )

    # Crée le graphe ONNX
    graph = onnx.helper.make_graph(
        nodes=[softmax_node],
        name=f"Softmax_axis_{axis}_rank_{rank}",
        inputs=[input_tensor],
        outputs=[softmax_output]
    )

    # Crée le modèle ONNX (opset 13)
    model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)])
    onnx.checker.check_model(model)
    onnx.save(model, f"softmax_axis_{axis}_rank_{rank}.onnx")
    return ort.InferenceSession(f"softmax_axis_{axis}_rank_{rank}.onnx")

def do_softmax(x, axis=1):
    """
    Effectue le softmax sur le tenseur x le long de l'axe spécifié.
    Crée une session ONNX adaptée au rang du tenseur et à l'axe.
    """
    rank = x.ndim
    session = create_softmax_model(axis=axis, rank=rank)
    output = session.run(None, {input_name: x})

    x_f = np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '')
    o_f = np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '')

    print(f"X (axis={axis}) = {x_f}")
    print(f"Softmax(X) = {o_f}\n")

# Options d'affichage pour numpy
np.set_printoptions(precision=8, floatmode='fixed')

# Exemple avec un tenseur 3D
x3d = np.array([
    [[1, 2, 3], [4, 5, 6]],
    [[10, 20, 30], [40, 50, 60]]
], dtype=np.float32)

do_softmax(x3d, axis=2)  # Softmax sur la dernière dimension

X (axis=2) = [[[ 1.00000000, 2.00000000, 3.00000000],  [ 4.00000000, 5.00000000, 6.00000000]], [[10.00000000,20.00000000,30.00000000],  [40.00000000,50.00000000,60.00000000]]]
Softmax(X) = [[[9.00305733e-02,2.44728476e-01,6.65240943e-01],  [9.00305733e-02,2.44728476e-01,6.65240943e-01]], [[2.06105999e-09,4.53978682e-05,9.99954581e-01],  [2.06105999e-09,4.53978682e-05,9.99954581e-01]]]

